In [1]:
import cv2 #gerekli kütüphaneleri (paketleri) aktaralım
import matplotlib.pyplot as plt
from collections import deque
import numpy as np

buffer_size = 16 #nesne merkezini depoloyacak veri tipi 
pts = deque(maxlen=buffer_size)  

lowerBlue = (84,98,0)  #alt ve üst mavi değer aralıklarını belirledik
upperBlue = (179,255,255)

cap = cv2.VideoCapture(0) #webcamden görüntüyü al 
cap.set(3,960) #genişlik boyutu 960
cap.set(4,480) #yükseklik boyutunu ise 480 yapmış olduk

while True: #frameleri okumak için sonsuz bir döngü oluşturdul
    ret,frame = cap.read()
    frame = cv2.flip(frame,1) #görüntüyü y eksenine göre çevir ve ayna görüntüsü elde et
    if ret == False:  #görüntüyü okuyamazsak döngüyü kır
        break

    blurred = cv2.GaussianBlur(frame,(11,11),0) #gürültüler için blurladık

    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)  #maskelemek için hsv formatına çevirdik

    mask = cv2.inRange(hsv,lowerBlue,upperBlue)  #mask oluşturduk

    mask = cv2.erode(mask,None,iterations = 2)  #mavi cismi bulurken kenarlardaki küçük bozulmalara karşın morfolojik işlemlerden geçirdik
    mask = cv2.dilate(mask,None,iterations = 2) 
    

    cnts,_ = cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)  #mavi cismin konturlarını bulduk
    center = None

    if len(cnts) > 0 :
        c = max(cnts,key=cv2.contourArea)
        rect = cv2.minAreaRect(c) # o konturu kapsayan minimim dikdörtgeni döndürür
        
        ((x,y),(width,height),rotation) = rect
        s = "x :{}   y : {} width :{} height : {} rotation : {}".format(np.round(x),np.round(y),np.round(width),np.round(height),np.round(rotation)) #int değerleri yuvarlayarak yazdırdık
        print(s)

        box = cv2.boxPoints(rect) #kutucuk
        box = np.int64(box)

        #konturun merkezini buluruz
        M = cv2.moments(c)
        center = (int(M["m10"]/M["m00"]),int(M["m01"]/M["m00"]))

        cv2.drawContours(frame,[box],0,(0,255,255),2) # konturu kapsayan en küçük dikdörtgeni çizdirdik
        cv2.circle(frame,center,5,(255,0,255),-1) #konturun merkezine içi dolu  bir çember çizdik
    
        cv2.putText(frame,s,(15,25),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,0),2) 
    
    pts.appendleft(center) #noktalarımı kuyruğa ekliyoruz
    for i in range(1,len(pts)):  #geçmişteki son 16(buffer_size) noktayı görebiliyoruz bu sayede 
        if pts[i-1] is None or pts[i] is None: #nesnenin geçmişte nerede oldugunu anlayabiliyoruz
            continue
        cv2.line(frame,pts[i-1],pts[i],(0,255,0),3) 

    cv2.imshow('mask',mask) #mask ve frame'imizi imshowluyourz
    cv2.imshow('frame',frame)

    if cv2.waitKey(1) == ord('q'): #q ya basılınca döngüyü kır
        break

cap.release()  #görüntüyü serbest bırakıyoruz
cv2.destroyAllWindows() #tüm pencereleri kapatıyourz
